<p style="font-weight:bold;"> <span style="font-size: 36px"> Reports</span> </p>

<a id='report-production'></a>

# Infrastructure and Configuration

## Initialize data

In [ ]:
/* DataSource is configured and connected to real database */
//#!eval-notebook "../Database/Configure"

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
//#!import "ReportsNewWay"

In [ ]:
Workspace.InitializeFrom(DataSource);

## Imports and Configurations

In [ ]:
var reportStorage = new ReportStorage(Workspace, Report);
await reportStorage.InitializeReportIndependentCacheAsync();
var mostRecentPartition = (await Workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();
var reportingNodeRoot = (await Workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName;
await reportStorage.InitializeAsync((mostRecentPartition.Year, mostRecentPartition.Month), reportingNodeRoot, null, CurrencyType.Contractual);
var identities = reportStorage.GetIdentities((mostRecentPartition.Year, mostRecentPartition.Month), reportingNodeRoot, null, CurrencyType.Contractual);

## Calling Scope

In [ ]:
var universe = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

# Report Settings and Storage Update

In [ ]:
((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) Args =
    //((2020, 12), reportingNodeRoot, null,CurrencyType.Contractual)
    ((2021, 3), reportingNodeRoot, null,CurrencyType.Contractual)
    //((2021, 3), reportingNodeRoot, null,CurrencyType.Functional)
    //((2021, 3), reportingNodeRoot, null,CurrencyType.Group)
     ;

await reportStorage.InitializeAsync(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);
identities = reportStorage.GetIdentities(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);

# Use cases

For demostration purposes we import data for 7 *Group of Insurance Contract* (GIC) and 4 *Group of Reinsurance Contract* (GRIC). 
<br> Data set consists in cashflows, actuals, and parameters.



# Present Value

Present values of the best estimate of future cashflows are shown here in an Analysis of Change report.

The report view can be modified with the Slice options for the columns by changing the <code>SliceColumnBy</code> inputs in the next command cell.
For example one can add <code>"GroupOfContract"</code> to separate the contributions of the individual Group of Contracts.
<br> We suggest to add this slice between the <code>"LiabilityType"</code> and the <code>"EconomicBasis"</code> as the the order of the inputs corresponds to the order of the columns shown in the report to expand the data.

Aggregated values are displayed when the data has a finer granularity than the one selected by the report slice options.

In [ ]:
Report.ForDataCube((universe.GetScopes<LockedBestEstimate>(identities).Aggregate().LockedBestEstimate + universe.GetScopes<CurrentBestEstimate>(identities).Aggregate().CurrentBestEstimate)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType", "EconomicBasis") //"GroupOfContract"
    .ReportGridOptions()
    .ToReport()

# Risk Adjustment

In [ ]:
Report.ForDataCube((universe.GetScopes<LockedRiskAdjustment>(identities).Aggregate().LockedRiskAdjustment + universe.GetScopes<CurrentRiskAdjustment>(identities).Aggregate().CurrentRiskAdjustment)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "EconomicBasis", "LiabilityType","GroupOfContract")
    .ReportGridOptions()
    .ToReport()

# Actuals

In [ ]:
Report.ForDataCube(universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Written
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("AmountType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract")
    .ReportGridOptions(reportHeight: 450)
    .ToReport()

## Advance, Overdue Actuals

In [ ]:
Report.ForDataCube((universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Advance + universe.GetScopes<WrittenAndAccruals>(identities).Aggregate().Overdue)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract", "EstimateType")
    .ReportGridOptions(reportHeight: 400)
    .ToReport()

## Deferrable Actuals

In [ ]:
Report.ForDataCube(universe.GetScopes<Deferrals>(identities).Aggregate().Deferrals
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract")
    .ReportGridOptions(reportHeight: 400)
    .ToReport()

# Fulfilment Cashflow

In [ ]:
Report.ForDataCube(universe.GetScopes<Fcf>(identities).Aggregate().Fcf    
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract", "EconomicBasis")//, "EstimateType")//, "AmountType")
    .ReportGridOptions()
    .ToReport()

# Actuarial Experience Adjustment

In [ ]:
Report.ForDataCube(universe.GetScopes<ExperienceAdjustment>(identities).Aggregate().ActuarialExperienceAdjustment
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("EstimateType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "GroupOfContract", "AmountType" )//"LiabilityType", "EstimateType")
    .ReportGridOptions(reportHeight: 300, headerColumnWidth: 300)
    .ToReport() 

# LRC Technical Margin

In [ ]:
Report.ForDataCube(universe.GetScopes<LrcTechnicalMargin>(identities).Aggregate().LrcTechnicalMargin
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "GroupOfContract")
    .ReportGridOptions(reportHeight: 600)
    .ToReport()

# Contractual Service Margin / Loss Component / Loss Recovery Component

In [ ]:
Report.ForDataCube((universe.GetScopes<Csm>(identities).Aggregate().Csm + universe.GetScopes<Lc>(identities).Aggregate().Lc + universe.GetScopes<Loreco>(identities).Aggregate().Loreco)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "GroupOfContract", "EstimateType")
    .ReportGridOptions()
    .ToReport()

# LRC Actuarial

In [ ]:
Report.ForDataCube((universe.GetScopes<LrcActuarial>(identities).Aggregate().LrcActuarial)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), /*"LiabilityType","GroupOfContract",*/ "EstimateType")
    .ReportGridOptions()
    .ToReport()

# LRC

In [ ]:
Report.ForDataCube((universe.GetScopes<Lrc>(identities).Aggregate().Lrc)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType),"GroupOfContract", "EstimateType")
    .ReportGridOptions(300)
    .ToReport()

# LIC Actuarial

In [ ]:
Report.ForDataCube((universe.GetScopes<LicActuarial>(identities).Aggregate().LicActuarial)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("Novelty","VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), /*"GroupOfContract",*/ "EstimateType")
    .ReportGridOptions()
    .ToReport()

# LIC

In [ ]:
Report.ForDataCube((universe.GetScopes<Lic>(identities).Aggregate().Lic)
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), /*"GroupOfContract",*/ "EstimateType")
    .ReportGridOptions(300)
    .ToReport()

# Financial Performance

Use the expand and collapse buttons in the report rows to change the granularity of the figures displayed.

In [ ]:
Report.ForDataCube(universe.GetScopes<FinancialPerformance>(identities).Aggregate().FinancialPerformance
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType", "EstimateType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract")
    .ReportGridOptions(reportHeight: 900, headerColumnWidth: 500, groupDefaultExpanded: 3)
    .ToReport()